# Learning Objectives


- Practice with http.server Python's module.
- Use encode() to send data from the server.
- Explain queries and quoting as a way to send data in a request.
- Differentiate a GET Request and a POST Request.
- Demonstrate Post-Redirect-Get
- Define JSON

# Python Strings

The `str` class in python represents strings as unicode. This is a decoded string. Unicode has enough space to handle a massive amount of characters (1,114,112 codepoints), but currently only 137,993 codepoints have been defined. To display strings on the web, they need to be encoded into bytes. Common encodings include ASCII, UTF-8 and UTF-16.

In [1]:
import math

math.log(1114112, 2)

20.087462841250343

In [1]:
>>> from urllib.parse import urlparse, parse_qs
>>> address = 'https://www.google.com/search?q=gray+squirrel&tbm=isch'
>>> parts = urlparse(address)
>>> print(parts)

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='q=gray+squirrel&tbm=isch', fragment='')


In [3]:
parts.query

'q=gray+squirrel&tbm=isch'

In [4]:
parse_qs(parts.query)

{'q': ['gray squirrel'], 'tbm': ['isch']}

In [4]:
2**20

1048576

In [ ]:
import http.server

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def __init__(self, args):
        http.server.BaseHTTPRequestHandler.init(args)
        
    def do_GET(self, request):
        
        
server_instance = http.server.HTTPServer(RequestHand)

# Queries and quoting

### Unpacking query parameters
When you take a look at a URI for a major web service, you'll often see several query parameters, which are a sort of variable assignment that occurs after a ? in the URI. For instance, here's a Google Image Search URI:

https://www.google.com/search?q=gray+squirrel&tbm=isch

The **query** part of the URI is the part after the ? mark. Conventionally, query parameters are written as key=value and separated by & signs. So the above URI has two query parameters, q and tbm, with the values gray+squirrel and isch.

There is a Python library called `urllib.parse` that knows how to unpack query parameters and other parts of an HTTP URL. (The library doesn't work on all URIs, only on some URLs.) Take a look at the urllib.parse documentation [here](https://docs.python.org/3/library/urllib.parse.html). Check out the `urlparse` and `parse_qs` functions specifically.

### URL quoting
Did you notice that `'gray+squirrel'` in the query string became `'gray squirrel'` in the output of `parse_qs`?

HTTP URLs aren't allowed to contain spaces or certain other characters. So if you want to send these characters in an HTTP request, they have to be translated into a "URL-safe" or "URL-quoted" format.

"Quoting" in this sense doesn't have to do with quotation marks, the kind you find around Python strings. It means translating a string into a form that doesn't have any special characters in it, but in a way that can be reversed (unquoted) later.

(And if that isn't confusing enough, it's sometimes also referred to as URL-encoding or URL-escaping).

One of the features of the URL-quoted format is that spaces are sometimes translated into plus signs. Other special characters are translated into hexadecimal codes that begin with the percent sign.

Take a look at the [documentation for urllib.parse.quote and related functions](https://docs.python.org/3/library/urllib.parse.html#url-quoting).

Later in the course when you want to construct a URI in your code, you'll need to use appropriate quoting. More generally, whenever you're working on a web application and you find spaces or percent-signs in places you don't expect them to be, it means that something needs to be quoted or unquoted.

# GET and POST

### Form methods: GET and POST
In the last lesson, I mentioned that `GET` is only one of many HTTP verbs or methods.

When a browser submits a form via `GET`, it puts all of the form fields into the URI that it sends to the server. These are sent as a query, in the request path — just like search engines do. They're all jammed together into a single line. Since they're in the URI, the user can bookmark the resulting page, reload it, and so forth.

This is fine for search engine queries, but it's not quite what we would want for (say) a form that adds an item to your shopping cart on an e-commerce site, or posts a new message on a comments board. `GET` methods are good for search forms and other actions that are intended to look something up or ask the server for a copy of some resource. But `GET` is not recommended for actions that are intended to alter or create a resource. For this sort of action, HTTP has a different verb, `POST`.

### Idempotence
Vocabulary word of the day: **idempotent**. An action is idempotent if doing it twice (or more) produces the same result as doing it once. "Show me the search results for 'polar bear'" is an idempotent action, because doing it a second time just shows you the same results. "Add a polar bear to my shopping cart" is not, because if you do it twice, you end up with two polar bears.

`POST` requests are not idempotent. If you've ever seen a warning from your browser asking you if you really mean to resubmit a form, what it's really asking is if you want to do a non-idempotent action a second time.

(Important note if you're ever asked about this in a job interview: idempotent is pronounced like "eye-dem-poe-tent", or rhyming with "Hide 'em, Joe Tent" — not like "id impotent".)


# A server for POST
In the next few exercises, you'll be building a **messageboard server**.

When a user goes to the main page in their browser, it'll display a form for writing messages, as well as a list of the previously written messages.

Submitting the form will send a request to the server, which stores the submitted message and then re-displays the main page.

In order to test your **messageboard server**, you'll need to install the `requests` module, which is a Python module for making HTTP requests.

We'll see much more about this module later in this lesson. For now, just run `pip3 install requests` in your terminal to install it.

### Why don't we want to use GET for submitting the form?
Imagine if a user did this. They write a message and press the submit button … and the message text shows up in their URL bar. If they press reload, it sends the message again. If they bookmark that URL and go back to it, it sends the message again.

This doesn't seem like such a great experience. So we'll use POST for message submission, and GET to display the main page.

### POST handlers read the request body
Previously in the HelloServer exercise, you've written handler classes that have just a single method, do_GET.

But a handler class can have do_POST as well, to support GET and POST requests. This is exactly how the messageboard server will work.

When a GET request comes in, the server will send the HTML form and current messages. When a POST request comes in with a new message, the server will store the message in a list, and then return all the messages it's seen so far.

The code for a do_POST method will need to do some pretty different things from a do_GET method. When we're handling a GET request, all the user data in the request is in the URI path. But in a POST request, it's in the request body.

Inside do_POST, our code can read the request body by calling the self.rfile.read method. self.rfile is a file object, like the self.wfile we saw earlier — but rfile is for reading the request, rather than writing the response.

However, self.rfile.read needs to be told how many bytes to read … in other words, how long the request body is.

### Headers are strings (or missing)
The handler class gives us access to the HTTP headers as the instance variable self.headers, which is an object that acts a lot like a Python dictionary. The keys of this dictionary are the header names, but they're case-insensitive: it doesn't matter if you look up 'content-length' or 'Content-Length'. The values in this dictionary are strings: if the request body is 140 bytes long, the value of the Content-length entry will be the string "140". We need to call self.rfile.read(140) to read 140 bytes; so once we read the header, we'll need to convert it to an integer.

But in an HTTP request, it's also possible that the body will be empty, in which case the browser might not send a Content-length header at all. This means we have to be a little careful when accessing the headers from the self.headers object. If we do self.headers['content-length'] and there's no such header, our code will crash with a KeyError. Instead, we'll use the .get dictionary method to get the header value safely.

So here's a little bit of code that can go in the do_POST handler to find the length of the request body and read it:

`
length = int(self.headers.get('Content-length', 0))
data = self.rfile.read(length).decode()
`

Once you read the message body, you can use urllib.parse.parse_qs to extract the POST parameters from it.

With that, you can now build a do_POST method!

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />